# How to read this document

This document is designed to showcase the different basic functionality of Hyperspy for the analysis of EELS data. This notebook is split in five main parts:

#### I. Introduction to Hyperspy

#### II. Data visualisation

#### III. Core-loss analysis

#### IV. Los-loss analysis

#### V. Machine learning

**The jupyter notebook extensions are installed. Thus, you can navigate between sections using the left panel**

Not everything is intended to be read the from the first time. Optional comments along the document are marked by emojis: 
- ⭐ This is a tip, describing more in depth the inner workings of hyperspy.
- 🏠 It's an optional comment. Something that you may not use here but that you can use back to your lab.

## Using Jupyter

- Autocompletion: When you are typing code and ask for a function of an object (e.g., `data.`), just after typing the dot you can press tab to see the possible options. You can continue typing to reduce the number of possibilities.

- Contextual help: I recommend that you have a quick acces to the documentation opened in a tab. Go to the top menu bar and click on "Help --> Show Contextual Help". Positioning you pointer next to a code part will show you its associated documentation.

- Inline help: If the above does not work you can also type a `?` at the end of a function (e.g., `data.function?`)

# I. Introduction to Hyperspy

![image](images/hyperspy_logo.png)

Hyperspy is a tool for the visualisation and manipulation of high-dimensional data. The main features of Hyperspy are the following:
- The Signal1D and Signal2D objects
- Model fitting
- Machine learning

Over the years, the hyperspy community has built a rich environment of domain-specific packages for hyperspy-based data analysis. In this notebook, we will use exspy:

![image](images/exspy_logo.png)

A large part of this practical is inspired from the hyperspy documentation : http://hyperspy.org/hyperspy-doc/current/index.html

## Import Hyperspy

⭐ We choose first a graphical backend that manage interactivity (```%matplotlib qt```).

⭐ We can cancel the appearance of warnings for better readability (```warnings.filterwarnings('ignore')```).

In python the libraries have to be imported to use their functions. For clarity, hyperspy will be imported with the name ```hs```. 

In [ ]:
%matplotlib qt

import warnings
warnings.filterwarnings('ignore')

import hyperspy.api as hs

## Loading data

Hyperspy is able to load and manage data from many different open-source and proprietary formats, such as: 
- dm3, dm4
- emd, bcf
- tiff, jpg, etc...

For an exhaustive list : http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#supported-formats

⭐ Without arguments ```hs.load()```  makes a file explorer window pop up. You can also load a list of files.

🏠 Depending on the file format it loads either as a directly usable dataset (e.g. dm3, dm4) or as a list of datasets (e.g. bcf, emd). You can then work on the elements of the loaded list.

The data below were kindly provided by: **Maya Marinova**.

In [ ]:
low_loss = hs.load('data/core_loss/EELS Spectrum Image (low-loss) (dark ref corrected).dm4')
core_loss = hs.load('data/core_loss/EELS Spectrum Image (high-loss) (dark ref corrected).dm4')
adf = hs.load("data/core_loss/Gatan BF_DF.dm4")
adf_survey = hs.load("data/core_loss/SI Survey Image.dm4")

## Metadata

The loaded data also contain metadata. We will see that some of these metadata are used for specific functions (e.g. beam energy), such as modelling. In this tutorial all the relevant metatadata is already filled in.


In [ ]:
core_loss.metadata

### Orginal metadata

Sometimes the import fails to retrieve important metadata. They can be inspected using the `original_metadata` attribute.

In [ ]:
core_loss.original_metadata

### Modify metadata

You can modifiy any field of the metadata to suit your needs in two ways:
- Using helper functions such as `set_microscope_parameters`
- Directly modifying the entries

In [ ]:
core_loss.set_microscope_parameters(beam_energy=200)
core_loss.add_elements(['Fe', 'Ni'])
core_loss.metadata.General.experimentalist = "Maya Marinova"
core_loss.metadata

## Signals

### navigation and signal dimensions

The key feature of hyperspy are the Signal objects. They are based on the concept of separating array dimensions between navigation and signal. The navigation axes are considered as sampling of the collected signal. **Hence, not all axes are treated equally, which enables an easier handling of high dimensional data.** Most Hyperspy functions operate of the signal axes and iterate over the navigation axes. For instance, when calling `signal.mean()`, it will, by default, return the mean of the signal over the navigation dimension. 

In STEM-EELS spectrum-imaging, EELS spectra contain the signal of interest that are sampled over an area. In this case, X and Y are the navigation axes and E is the signal axis. Calling `signal.mean()` will thus conviniently return the mean EELS spectrum.

The notation is as follows: `(navigation axes| signal axes>`.

Signals axes are either 1D or 2D as it is usually a convinient representation of signals. 

⭐ You can change the axes from signal to navigation using the transpose operation : `signal.T`

In [ ]:
core_loss

In [ ]:
core_loss.mean()

### axes_manager

The different axes, their calibration and properties are stored in the `axes_manager` attribute. A Graphical User Interface (gui), is provided by Hyperspy  

🏠 You can bypass the gui using `signal.axes_manager.navigation_axes[...]` or `signal.axes_manager.signal_axes[...]`. You can set their properties as `signal.axes_manager.navigation_axes.set(name=("X", "Y"), offset=10, units="nm")`. Once the axes have a name you can also acces the axes directly: `signal.axes_manager['X']`.

⭐ Hyperspy can also handle non-linear axes. See DataAxis or FunctionalDataAxis in Hyperspy documentation.

In [ ]:
core_loss.axes_manager

In [ ]:
core_loss.axes_manager.gui()

### Slicing data

The signals also provide a convinient way to slice data according to your needs. You can slice either along the navigation axes using `signal.inav` or along the signals axes using `signal.isig`. The syntax is then the following: 
```
signal.inav[start:stop:step]
```

Note that `start`, `stop` and `step` can be floats or string, expressed in calibrated units. By default, `start` = 0, `stop` = -1 and `step` = 1.

In [ ]:
print("Slice using array indices : ",core_loss.isig[670:850])
print("Slice using calibrated values : ",core_loss.isig[670.0:850.0:0.25])
print("Slice one over two values : ",core_loss.isig[670.0:850.0:0.5])
print("Slice using eVs : ",core_loss.isig["670.0 eV":"850.0 eV":"0.25eV"])
print("Slice using meVs : ",core_loss.isig["670000 meV":"850000 meV":"250meV"])

### Signal types

The signals are usually from of a child-type of `Signal1D` or `Signal2D`, that originate from domain-specific python packages directly linked to Hyperspy. This is illustrated just below:

![image](images/HyperSpySignalOverview.png)

Each child-type implements domain-specific functions for data analysis. For instance, in this notebook we will take advantage of functions specific to the analysis of EELS spectra using the `EELSSpectrum` object.

Here is a link to the list of Hyperspy extensions:
[List of Hyperspy extensions](https://github.com/hyperspy/hyperspy-extensions-list)

⭐⭐ You can code your own extensions, see the online tutorial: [How to write an extension](https://hyperspy.org/hyperspy-doc/current/dev_guide/writing_extensions.html)

#### Printing the installed child-signals

In [ ]:
hs.print_known_signal_types()

# II. Data visualisation

Hyperspy also provides gui elements for data visualisation. Executing the cell below enables you to browse the data in similar way as with DigitalMicrograph.

In [ ]:
# Create a ROI object. Without arguments it will use default coordinates, that are usually working well.
roi = hs.roi.RectangularROI()

# Plotting the different datasets
low_loss.plot()
core_loss.plot()
adf.plot()

# Linking the ROI with the datasets
ll_select = roi.interactive(low_loss)
cl_select = roi.interactive(core_loss)
roi.interactive(adf)

# Summing over the spectrum images
ll_spectrum = hs.interactive(ll_select.mean)
cl_spectrum = hs.interactive(cl_select.mean)

# Plotting the spectra
ll_spectrum.plot()
cl_spectrum.plot()

# III. Core-loss analysis

🏠 In this practical we will mostly use the gui elements of Hyperspy, which in general, work the following way:
- If it is an attribute such as `signal.axes_manager` you can call the `signal.axes_manager.gui()` function.
- If it is a function such as `signal.axes_manager.remove_background()` you can call it without arguments, and it will activate the gui. You can still call the function with arguments (Check the contextual help to know which arguments to use).
Everything shown here also work without the gui elements.

🏠 Most gui elements of Hyperspy provide a corresponding 'Help' section. 

## Pre-processing

### (Optional) Calibration

The `signal.calibrate()` function works similarly as in Digital Micrograph. You can select a range on a picked spectrum with identifiable features and apply the calibration. This overwrites the `axes_manager` values and works on the signal axis. 

In [ ]:
core_loss.calibrate()

### X-ray removal

Hyperspy also provides a convinient way to remove cosmic X-rays from the spectrum image: the function `spikes_removal_tool`. Note that, this procedure overwrites the data.

In [ ]:
core_loss.spikes_removal_tool()

### Align energy

We can first plot the estimated zero-loss peak energy position and check if the estimation seems correct. The `estimate_zero_loss_peak_centre` function returns a signal that we need then to plot. This step is optional.

In [ ]:
low_loss.estimate_zero_loss_peak_centre().plot()

If the estimation seems correct, we can apply the energy shift correction using the `align_zero_loss_peak` function. The energy shift is also applied to a list of other signals with the same dimensions using the `also_align` keyword argument.

🏠 If you have a peak with higher intensity than the zero-loss it is best to precise the energy range over which to align in energy. 

In [ ]:
low_loss.align_zero_loss_peak(also_align=[core_loss], signal_range=[-5.0,5.0])

### Remove plural scattering

The electrons can undergo several inelastic scattering events during their travel inside of the sample. However we want to access the single scattering distribution which directly correspond to a physically meaningfull spectrum (as direct measure of transitions probabilty). The single scattering distribution is given by : 

$$ S_{1}(E) = I_{0} t \theta(E) * \sigma(E) $$

Where $I_{0}$ is the incoming beam intensity, $t$ is the thickness of the sample, $\theta(E)$ is the zero loss peak and $\sigma(E)$ is inelastic scattering spectrum. The zero loss peak is already convolved to the spectrum. Thus, even in the case of single scattering, the deconvolution can improve the energy resolution. When the sample is too thick, the spectrum is dominated by the multiple scattering distribution and $\sigma(E)$ can not be straightforwardly obtained.

For example, the double scattering distribution is given by : 

$$ S_{2}(E) = I_{0} \frac{t^{2}}{2} \theta(E) * \sigma(E) * \sigma(E) $$

In that case, it is important to perform the deconvolution with the full low loss spectrum. 

#### Thickness estimation

We can first estimate the thickness of the sample with the `estimate_thickness` function that computes the relative thickness over a low loss EELS dataset. Since it outputs a Signal2D, we use the `plot` function to inspect the results.

The first argument is the energy threshold below which the signal is integrated to calculate the thickness. 

In [ ]:
low_loss.estimate_thickness(5.0).plot()

#### Deconvolution

The Richardson-Lucy algorithm can be used to deconvolve the effect of plural scattering from the core-loss. It is usually the prefered deconvolution method in EELS. The low-loss is used through the `psf` keyword argument. The number of iterations (`iterations =`) is a key parameter here, depending on your dataset you should increase it or decrease it. If you use too many iterations, you will see ripples artefacts appear.

🏠 See the "gloter2003_eels_deconvolution.pdf" for more information on deconvolution in EELS.

In [ ]:
core_loss.richardson_lucy_deconvolution(psf=low_loss,iterations = 20)

## Edge identification and extraction

Hyperspy has a dedicated tool for edge identification that gives you all the possible edge at the selected energy range. 

In [ ]:
core_loss.edges_at_energy()

### Extracting edges

The EELSSpectrum object has an implementation of power law background removal for edge extraction. The instruction on how to use the function are described during execution. Here we apply the background removal on a single spectrum (extracted above).

For EELS, the Power Law background is the most adapted.

In [ ]:
cl_spectrum.remove_background()

## Elemental mapping

While there is a helper function in Energy-Dispersive X-ray Spectroscopy for elemental mapping, there is no such functionalit for EELS. That is why we implement the functionality in several steps: 
- Make a copy of the data to try several background removals
- Remove the background of the full core-loss spectrum-image

In [ ]:
cl_copy = core_loss.deepcopy()
cl_copy.remove_background()

- We then switch from Signal1D to Signal2D to enable easier interactive mapping. This would be the data equivalent of switch from a STEM-EELS spectrum-image acquisition to EFTEM spectrum-image acquisition.
- We can then apply a `SpanROI` to select and map an energy range. The syntax is very similar to the custom data picker tool that we have seen above. 

In [ ]:
cl_eftem = cl_copy.T
roi = hs.roi.SpanROI()
cl_eftem.plot()
energy_span = roi.interactive(cl_eftem)
elemental_map = hs.interactive(energy_span.mean)
elemental_map.plot()

- To get nice maps, it is better to perform a background removal for each edge. Just below we can save the resulting map.

In [ ]:
elemental_map.save("NiL_845_to_854.hspy")

#### 🏠 Converting `.hspy` data to Digital Micrograph (DM) readable format

You first need to install this plugin in DM : [ripple plugin](http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#importrpl-digital-micrograph-plugin)

Then you just need to save your data in `.rpl` format using hyperspy. It will produce `.rpl` and `.raw` files. Using the DM plugin you'll be able to open the `.rpl` file and read your data in DM.

In [ ]:
elemental_map.save("NiL_845_to_854.rpl")

## Core-loss modelling

Hyperspy has advanced model fitting features that:
- use domain-specific knowledge and methods based on its signal child-type
- use metadata as model parameters
- gui elements for initilisation, etc ...

You can save the model with the dataset.

### Important arguments 

The `signal.create_model` function takes a few important arguments in EELS:
- `GOS=` stands for Generalised Oscillator Strangth and we can use `"dirac"`, `"dft"`, `"Hartree-Slater"` or `"hydrogenic"`. The Hyperspy documentation recommends the use of `"dft"` for white lines and C, N and O edges. Relativistic effects are included using `"dirac"`.
- `low_loss=` is the argument to include plural scattering correction

More information on the `"dft"` and `"dirac"` related databases, see [dft DB](doi:10.5281/zenodo.7645765) and [dirac DB](doi:10.5281/zenodo.12800856).

### Model options

An exhaustive list of model functions is available in exspy or hyperspy documentation.

- In most cases, it is important to enable the fine structure of core-loss edges for more accurate fitting. This is done using: `model.enable_fine_structure()`

- In some cases, the valence of elements can change compared to the tabulated one. It will change the energy of the edge onset (this shift is usually called a chemical shift). You can use the function `model.enable_free_onset_energy()` to also fit the energy onset value. This feature can greatly help you if your energy scale is not fully accurate.

- If you removed the background beforehand you should disable the background component with `model.disable_background()`

### Single spectrum fitting

While it is possible to fit STEM-EELS spectrum images, pixel by pixel, it is much faster to first test on a single spectrum. We are mostly interested in the average Ni/Fe ratio, that's why we cut the signal range to 50 eV before the Fe L3 edge.

In [ ]:
cl_mean = core_loss.mean()
model = cl_mean.create_model(GOS = "dirac",low_loss = low_loss.mean())
model.enable_fine_structure()
model.set_signal_range(650.0,950.0)

### Manually tuning initialisation

Using the gui elements, it is possible to manually initialise the fitting parameters and add bounds to the parameters.

I recommend using this interface only if the fit doesn't produce a valid result with the automatic initialisation. 

In [ ]:
model.plot()
model.gui()

### Fitting

The `smart_fit` method is tailored for EELS core-loss fitting for better modelling.

The `fit` and `smart_fit` methods take two important arguments:
- `loss_function=` is the method that measure the discrepancies between the data and the model. In general, people use the sum of the quadratic differences as a loss function called "least square" (it is the default value: `ls`). It is a mathematically correct formulation for data presenting a gaussian noise with fixed variance.
  - Data with noise caracteristics with fixed variance is called "homoscedastic": 
  
  ![image](images/Homoscedasticity.png)

  - Data with noise caracteristics with variable variance (e.g. Poisson statistics) is called "heteroscedastic": 
  
  ![image](images/Heteroscedasticity.png)
 
   For most analysis the "least square" approach is good enough, however for a fully quantitative analysis it is better to use a tailored loss function.

- `optimizer=` is the algorithm used to minimise the loss function, i.e. fit the model to the data (Most of the algorithms are well suited for optimizing the least square loss function). Hyperspy proposes two kinds of algorithms:
  - "local" algorithm that can stop at a local minima of the loss function. 
  - "global" algorithm that can escape local minima by accepting temporarily parameter values that do not minimise the loss function. 

⭐ Hyperspy provides tools to estimate fitting errors on the parameters of the model, they can be accessed through the `std` value of fitted components.

Fitting is a vast subject, please check the documention of Hyperspy or ask your QEM teacher for more information about fitting.

In [ ]:
model.smart_fit()

### Plotting and printing the results

In [ ]:
model.plot()
model.quantify()

### Fitting spectrum images

You can apply the fitting procedure to a whole spectrum image to obtain quantified elemental maps (for example). The process is a bit slow and thus we are only sharing the code and the results we obtained. Here is the code: 

```python
# Load libraries
%matplotlib qt
import hyperspy.api as hs

# Load data
low_loss = hs.load('data/core_loss/EELS Spectrum Image (low-loss) (dark ref corrected).dm4')
core_loss = hs.load('data/core_loss/EELS Spectrum Image (high-loss) (dark ref corrected).dm4')
adf = hs.load("data/core_loss/Gatan BF_DF.dm4")
adf_survey = hs.load("data/core_loss/SI Survey Image.dm4")

# add metadata
core_loss.add_elements(['Fe', 'Ni','O'])

# Pre-process data
low_loss.align_zero_loss_peak(also_align=[core_loss], signal_range=[-5.0,5.0])

# Create the model
model = core_loss.create_model(GOS = "dirac",low_loss = low_loss)
model.enable_fine_structure()

# Fit
model.multifit(kind = "smart")

# Plot the results
model.plot_results()

# Save the model
model.save('Fe_Ni_O_model.hspy')
```

#### Fitted edge intensity


![image](images/Figure_intensity_parameter_of_Fe_L3_component_Signal.png)
![image](images/Figure_intensity_parameter_of_Ni_L3_component_Signal.png)
![image](images/Figure_intensity_parameter_of_O_K_component_Signal.png)

#### Elemental ratio maps
 
![image](images/Fe_Ni_ratio.png)
![image](images/Ni_ratio.png)
![image](images/Fe_O_ratio.png)
![image](images/O_Fe_ratio.png)

⚠️ Oxygen quantification must be treated with care:
- It is difficult to fit accurately.
- It easily comes from extrinsic sources.
- It is better estimated using charge balance (i.e. valence state study).

# IV. Low-loss analysis

In this next part of the tutorial, we will focus on STEM-EELS low-loss data acquired on a Ag nano-triangle on Si3N4 substrate. Surface plasmon modes appear at the surface of the triangle under the excitation of the electron beam. These modes can be mapped using STEM-EELS, see for example : "campos2017_plasmonic_mapping.pdf". These modes represent a local enhancement of the electric field that can be used, for example, for catalysis.  

This part of the notebook will help us answer the following questions:
- What kind of spectral features is my sample exhibiting ?
- What is the spatial distribution of these features ?
- At what energy, the spectral features are the most intense ?

The data pre-processing for low losses data shares some similarities with the above. Hence, we will go faster through the data loading and pre-processing, only emphasizing on the differences between low-loss and core-loss data analysis.

The data were kindly provided by: **Hugo Lourenço-Martins**.

## Loading data

In [ ]:
ll_spim = hs.load('data/low_loss/spim_low_loss.hspy')
ll_adf = hs.load('data/low_loss/ADF_low_loss.hspy')

## Pre-processing

### Energy alignement

In [ ]:
ll_spim.align_zero_loss_peak()

### Remove plural scattering

The nanotriangle sits on a Si3N4 membrane that has also an intense plasmonic response. That is why we extract a spectrum from the substrate far away from the triangle to deconvolve the Si3N4 contribution from the interesting signal. 

#### Extracting Si3N4 contribution

In [ ]:
# Create a ROI object. Without arguments it will use default coordinates, that are usually working well.
roi = hs.roi.RectangularROI()

# Plotting the different datasets
ll_spim.plot()
ll_adf.plot()

# Linking the ROI with the datasets
ll_select = roi.interactive(ll_spim)
roi.interactive(ll_adf)

# Summing over the spectrum images
ll_spectrum = hs.interactive(ll_select.mean)

# Plotting the spectra
ll_spectrum.plot()

#### Deconvolution

In [ ]:
ll_spim.richardson_lucy_deconvolution(psf = ll_spectrum, iterations=20)

### Plasmonic mapping

We can first manually extract plasmon maps by switching from `Signal1D` to `Signal2D`. 

In [ ]:
ll_eftem = ll_spim.T
roi = hs.roi.SpanROI()
ll_eftem.plot()
energy_span = roi.interactive(ll_eftem)
plasmon_map = hs.interactive(energy_span.mean)
plasmon_map.plot()

#### Saving maps

Don't forget to change the name each time.

In [ ]:
plasmon_map.save('plasmon_map_2eV.hspy')

# V. Machine learning

As stated above, the tools in the hands of the analyst are based on statistical priors on the data. It is even more important for machine learning tools such as Multivariate Statistical Analysis. Thus, in this section we will start by a non-exhaustive description of the statistics of EELS data.

## EELS statistics

The EELS processes are Poissonian in nature (see *EELS: EGERTON, Ray F. Electron energy-loss spectroscopy in the electron microscope. Springer Science & Business Media, 2011.*). Indeed, we observe a flow of electron over a given period of time and the inelastic scattering events are (in a good approximation) statistically independant from each other. It is often referred as shot noise.

Then the detection system can add up different kinds of noise such as beam jittering or dark current. We will detail here only the detector related noises.

### CCD detectors

The variance of the statistical noise of the CCD is given by the following formula : 

$$ Var(J(E)) \approx g + p J(E) $$

Where $J(E)$ is the detected electron current, following a Poisson statistics. $p$ is a conversion proportionality constant and $g$ is an additive gaussian noise originating from read-out noise and dark current. The variance is not constant over the channels of the detector.

### Direct detection

The direct electron detectors are mostly limited by shot noise, thus the noise variance becomes : 

$$ Var(J(E)) \approx p J(E) $$

The absence of dark noise ($g$) drastically improve the performances of these detectors (especially at low doses). The variance is still not constant over the channels of the detectors.

In general, the data acquired using direct detection are more straightforward to analyze.

#### References

*EGERTON, Ray F. Electron energy-loss spectroscopy in the electron microscope. Springer Science & Business Media, 2011.*

*De la Peña Manchón, Francisco J. Advanced methods for Electron Energy Loss Spectroscopy core-loss analysis, PhD thesis, 2010*

#### Notes

- **The methods presented here are very powerful but it may sometimes lead to wrong interpretations. If you plan to use them in your work we strongly recommend you to double check for the validity of the approach.** See for example: "lichtert2013_PCA_artefacts.pdf" in the references folder.

- For more in depth information about direct electron detectors : see "Hart2017_Direct_Detection_Electron_Energy-Loss_Spectroscopy.pdf" in the references folder.

## PCA

Principal Component Analysis is a machine learning algorithm that can be used for data analysis or for denoising of multidimensional data. It is based on statistical principles and it is used in a wide variety of domains from text analysis to meteorology and STEM-EELS is no exception.

### Principle

The data below are represented using two main axes, x and y. Each of those axes correspond to some variance of the data as it can be seen in the projection of the data on those axes (thick redlines). 

![image](images/random_data.png)

The PCA will reorganise the axes on which the data are represented. The axes are rotated so that they correspond to gradually decreasing variance of the data. It means that the first axis (arrow parallel the line) is represents the highest variance in the data (Thick blue line) and the second axis (arrow perpendicular to the line) represents lower variance (thick green line). 

![image](images/random_data_PCA.png)

For the data analysis, this reorganisation of the axes is useful. With the new representation we easily get that the main feature of the signal is the straight line and that the axis perpendicular to it represents mainly noise. **In general, after PCA you should determine which are the relevant axes (or components) describing your signal and discard the noisy ones.**

### The impact of noise statistics

In contrast to the data presented just above, the STEM-EELS data are heteroscedastic. The reorganisation of the axes will not occur in the same way. Therefore, a correction of this effect is required. That is also why it is important to understand the statistics of the data you analyze.

![image](images/poisson_data.png)

### Increasing the dimension of the data

The PCA principles apply even with data of higher dimension. For spectrum images, they can be represented as a collection of spectra (N pixels spectra). A point in 2D space corresponds to 2 coordinates (x,y). A point in 3D space corresponds to 3 coordinates (x,y,z). A spectrum can be seen as a point in E-dimensional space ($I_1, I_2, ..., I_E$).

![image](images/flat_spim.png)

PCA is going to decompose the data in two matrices. The first matricx called factors contains the vectors of the new representation, each column contains one spectrum-like axis. Each row of the second matrix (called loadings) correspond to the intensity of a given axis of the new representation. In the previous part with the line, the first line of the loadings will give out where each point is on the line. 

![image](images/decomposition.png)

For example, a spectral signature with high variance (such as the zero-loss peak) will have it's own axis in the new representation. Thus, there will be a corresponding loading describing the spatial evolution of the zero-loss peak intensity.

### Applying PCA

We use the `spim.decomposition` function to perform the PCA. Using the positional argument `spim.decomposition(True)`, the poissonian nature of the noise is taken into account (within some approximation). 

⭐ For fully taking into account the poisson statistics, you will need to use the maximum likelihood formulation of the algorihtm. This is more computationnally expensive though.

### References 

see "keenan2004_PCA_Poisson_normalisation.pdf" in the references folder.

In [ ]:
ll_spim.decomposition(True)

### Explained variance ratio

To help you determine the relevant components in your dataset, there is a tool in hyperspy to plot how much variance of the data correspond to each axis of the new representation. 

This plot organises the axes by decreasing variance. Often, this plot as an elbow shape. As a rule of thumb, the number of relevant components is approximately given by the position of the elbow.

In [ ]:
ll_spim.plot_explained_variance_ratio()

### Plotting the results

Hyperspy provides gui elements to plot the decomposition results. You can browse interactively through the loadings and factors.

In [ ]:
ll_spim.plot_decomposition_results()

You can also plot independantly the factors and the loadings using `spim.plot_decomposition_factors` and `spim.plot_decomposition_loadings`. Here are a few useful options for display: 

- You can use an integer `spim.plot_decomposition_factors(3)`, which will display the 3 first factors. Or you can use a list `spim.plot_decomposition_factors([1,3,5])` which will display the factors 1, 3 and 5 only.

- You can plot everything in separated windows using the `same_window = False` keyword argument

🏠 The syntax is really similar if you want to put the factors or loadings in a separate hyperspy signal with `spim.get_decomposition_factors` and `spim.get_decomposition_loadings`

In [ ]:
ll_spim.plot_decomposition_factors(3,same_window = False)

In [ ]:
ll_spim.plot_decomposition_loadings(3,same_window = False)

## NMF

### Principles

The Non-Negative Matrix Factorization is somewhat similar to PCA. It decomposes the data into factors and loadings, although they are not organized according to decreasing variance. The main difference is that the factors and loadings are constrained to positive values. 

We activate the NMF using the keyword argument `algorithm = 'NMF'`. It is an iterative algorithm and the calculation is much longer. That is why, We limit beforehand the number of components using the keyword argument `output_dimension = n`, where `n` is the integer you choose. The algorithm might not converge with the default amount of iterations, you can use `max_iter=` to increase it.

**The PCA modifies the spim object. It is recommended to reload the data before performing NMF.**

⭐ Hyperspy has a flexible enough interface. You can use many different decomposition algorithms such as the ones of scikit-learn with more or less the same syntax. For example, the value of `algorithm=` can be any object that implements the `fit` and `transform` methods.

In [ ]:
ll_spim.decomposition(True,algorithm = 'NMF',output_dimension = 20,max_iter = 2000)

In [ ]:
ll_spim.plot_decomposition_results()

## Understanding the results

The plasmon modes in nanotriangles can be divide in edge modes and pseudo-radial breathing modes (RBMs). RBMs have typically higher energy than edge modes. You can compare your extracted maps to the litterature.

![image](images/campos2017_maps.PNG)